In [1]:
!pip install timm
!pip install albumentations==0.4.6

     |████████████████████████████████| 431 kB 4.3 MB/s 
     |████████████████████████████████| 117 kB 4.1 MB/s 
     |████████████████████████████████| 948 kB 26.3 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=534d2b3f8240d637c7ae8c4d43a41cf9306383173c877f475b5bb6e2e522fa89
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import WeightedRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2
import time

In [3]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [6]:
train_png = sorted(glob('/content/drive/MyDrive/Colab Notebooks/dataset/train/*.png'))
test_png = sorted(glob('/content/drive/MyDrive/Colab Notebooks/dataset/test/*.png'))

In [7]:
train_y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/train_df.csv')
train_labels = train_y['label']

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key, value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [8]:
from albumentations.augmentations.transforms import RandomCrop
# Augmentation
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Rotate(25, interpolation=cv2.INTER_CUBIC),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388]),
    ToTensorV2(),
])

valid_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize([0.418256, 0.393101, 0.386632], [0.195055, 0.190053, 0.185323]),
    ToTensorV2(),                         
])

test_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize([0.418256, 0.393101, 0.386632], [0.195055, 0.190053, 0.185323]),
    ToTensorV2(),                           
])

In [9]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train', transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
        self.transform=transform
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
          augmented = self.transform(image=image)
          image = augmented['image']
        
        label = self.labels[idx]
        return image, label

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [10]:
# StratifiedKFold
folds = []
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, valid_idx in kf.split(train_png, train_labels):
  folds.append((train_idx, valid_idx))
fold=0
train_idx, valid_idx = folds[fold]

In [11]:
# Focal Loss
#def focalloss(pred, mask, gamma):
#    prob = torch.softmax(pred, 1)
#    mask_onehot = F.one_hot(mask, num_classes=88)
#    true_prob = prob[mask_onehot.type(torch.bool)]
#    celoss = -torch.log(true_prob)
#    weight = ((1-true_prob) ** gamma)
#    return (weight * celoss).mean()

In [12]:
batch_size = 25
epochs = 75

# Train
train_dataset = Custom_dataset(np.array(train_png)[train_idx], np.array(train_labels)[train_idx], mode='train', transform=train_transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=16)

# Validation
valid_dataset = Custom_dataset(np.array(train_png)[valid_idx], np.array(train_labels)[valid_idx], mode='valid', transform=valid_transform)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size, num_workers=16)

# Test
test_dataset = Custom_dataset(np.array(test_png), np.array(["tmp"]*len(test_png)), mode='test', transform=test_transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=16)

학습

In [13]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [14]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Focal Loss 적용시 주석처리
criterion = nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler() 

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)

        # Focal Loss 적용시 주석처리
        loss = criterion(pred, y)

        # Focal Loss 적용시 주석 해제
        #loss = focalloss(pred, y, 2)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    model.eval()
    valid_loss = 0
    valid_pred = []
    valid_y = []
    with torch.no_grad():
      for batch in (valid_loader):
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
          pred = model(x)

        # Focal Loss 적용시 주석처리
        loss = criterion(pred, y)
        
        # Focal Loss 적용시 주석 해제
        #loss = focalloss(pred, y, 2)
        valid_loss += loss.item() / len(valid_loader)
        valid_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        valid_y += y.detach().cpu().numpy().tolist()
      valid_f1 = score_function(valid_y, valid_pred)
    if valid_f1 >= best:
      best = valid_f1

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'VALID    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}    best : {best:.5f}')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b4_ra2_320-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_ra2_320-7eb33cd5.pth


epoch : 1/75    time : 391s/28922s
TRAIN    loss : 2.13761    f1 : 0.14597
VALID    loss : 0.95520    f1 : 0.15600    best : 0.15600
epoch : 2/75    time : 168s/12261s
TRAIN    loss : 0.81693    f1 : 0.17901
VALID    loss : 0.63518    f1 : 0.21408    best : 0.21408
epoch : 3/75    time : 168s/12072s
TRAIN    loss : 0.58007    f1 : 0.26452
VALID    loss : 0.46125    f1 : 0.29288    best : 0.29288
epoch : 4/75    time : 168s/11947s
TRAIN    loss : 0.46808    f1 : 0.36501
VALID    loss : 0.40656    f1 : 0.37779    best : 0.37779
epoch : 5/75    time : 168s/11731s
TRAIN    loss : 0.37833    f1 : 0.48629
VALID    loss : 0.35586    f1 : 0.44397    best : 0.44397
epoch : 6/75    time : 168s/11590s
TRAIN    loss : 0.33870    f1 : 0.49365
VALID    loss : 0.28861    f1 : 0.52187    best : 0.52187
epoch : 7/75    time : 168s/11436s
TRAIN    loss : 0.28426    f1 : 0.60756
VALID    loss : 0.28045    f1 : 0.54311    best : 0.54311
epoch : 8/75    time : 167s/11221s
TRAIN    loss : 0.24102    f1 : 0.

추론

In [15]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [16]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

제출

In [17]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [19]:
submission.to_csv("epoch75, batch25, drop_path, lr_1e-4, w_decay_1e-5, efficient4, StrKFold, Augmentation.csv", index = False)